## Example 2. Bi-Hamitonian 3D Lotka-Volterra System

This notebook solves the 3D bi Hamiltonian Lotka Volterra system of equations by different RK methods and their relaxation versions:

\begin{align}
    &\dot{u}_{1}=u_{1}\left(c u_{2}+u_{3}+\lambda\right) \\
    &\dot{u}_{2}=u_{2}\left(u_{1}+a u_{3}+\mu\right) \\
    &\dot{u}_{3}=u_{3}\left(b u_{1}+u_{2}+\nu\right) \;,
\end{align}
   where $\lambda, \mu, \nu >0$, $abc=-1$ and $\nu = \mu b - \lambda ab$. Domain is the interval $[0,400]$ and the parameters $(a,b,c,\lambda,\mu,\nu) = (-1,-1,-1,0,1,-1)$. The initial condition is taken as $\left(u_1(0),u_2(0),u_3(0)\right)^T = (1,1.9,0.5)^T$. Two nonlinear conserved quantities:

\begin{align}
    & H_1 = ab \ln{u_1} - b \ln{u_2} + \ln{u_3} \;, \\
    & H_2 = ab u_1 + u_2 - a u_3 + \nu \ln{u_2} - \mu \ln{u_3}.
\end{align}


In [ ]:
#Required libraries 
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from nodepy import rk
from IPython.display import clear_output
from RKSchemes import ssp22, heun33, ssp33, rk44, dp75

# Fifth order Fehlberg(6,5) method with a fourth order embeddings. The default Fehlberg method is 5th order
# with given b and 4th order with bhat. 
fehlberg45 = rk.loadRKM("Fehlberg45").__num__()

In [ ]:
# Required functions for the 3D Lotka-Volterra system
def LVS3D_f(u):
    u1 = u[0]; u2 = u[1]; u3 = u[2];
    a = -1; b = -1; c = -1; la = 0; mu = 1; nu = -1;
    du1 = u1*(c*u2 + u3 + la)
    du2 = u2*(u1 + a*u3 + mu)
    du3 = u3*(b*u1 + u2 + nu)
    return np.array([du1, du2, du3])

def H_1(u):
    u1 = u[0]; u2 = u[1]; u3 = u[2];
    a = -1; b = -1; c = -1; la = 0; mu = 1; nu = -1;
    return a*b*np.log(u1) - b*np.log(u2) + np.log(u3)

def H_2(u):
    u1 = u[0]; u2 = u[1]; u3 = u[2];
    a = -1; b = -1; c = -1; la = 0; mu = 1; nu = -1;
    return a*b*u1 + u2 -a*u3 + nu*np.log(u2) - mu*np.log(u3)

def rgam(gammas,u,inc1,inc2,E1_old,E2_old):
    gamma1, gamma2 = gammas
    uprop = u + gamma1*inc1 + gamma2*inc2  
    E1 = H_1(uprop)
    E2 = H_2(uprop)
    return np.array([E1-E1_old,E2-E2_old])

### Baseline RK methods

In [ ]:
# Compute solution with baseline methods
def compute_sol_without_relaxation(Mthdname,rkm, dt, f, T, u0,t0): 
    tt = np.zeros(1) 
    t = t0; tt[0] = t
    uu = np.zeros((1,np.size(u0))) 
    uu[0,:] = u0.copy()
    
    s = len(rkm)
    y = np.zeros((s,len(u0))) 
    F = np.zeros((s,len(u0))) 
    steps = 0
    
    while t < T and not np.isclose(t, T):
        clear_output(wait=True)
        if t + dt > T:
            dt = T - t
        for i in range(s):
            y[i,:] = uu[-1].copy()
            for j in range(i):
                y[i,:] += dt*rkm.A[i,j]*F[j,:]
            F[i,:] = f(y[i,:])
        inc = dt*sum([rkm.b[i]*F[i] for i in range(s)])    
        unew = uu[-1]+inc; t+= dt
        tt = np.append(tt, t)
        steps +=1
        uu = np.append(uu, np.reshape(unew.copy(), (1,len(unew))), axis=0)  
        print("Method = Baseline %s: Step number = %d (time = %1.2f)"%(Mthdname,steps,tt[-1]))
    return tt, uu

### Relaxation RK methods

In [ ]:
# Computing solution with multiple relaxation methods
def compute_sol_multi_relaxation(Mthdname,rkm, dt, f, T, u0, t0):
    tt = np.zeros(1) 
    t = t0; tt[0] = t
    uu = np.zeros((1,np.size(u0))) 
    uu[0,:] = u0.copy()
    
    s = len(rkm)
    y = np.zeros((s,len(u0))) 
    F = np.zeros((s,len(u0))) 
    
    G1 = np.array([]); G2 = np.array([])
    no_inv = 2; gammas0 = np.zeros(no_inv)
    
    errs = 0; steps = 0
    
    while t < T and not np.isclose(t, T):
        clear_output(wait=True)
        if t + dt > T:
            dt = T - t
        for i in range(s):
            y[i,:] = uu[-1].copy()
            for j in range(i):
                y[i,:] += dt*rkm.A[i,j]*F[j,:]
            F[i,:] = f(y[i,:])
            
        inc1 = dt*sum([rkm.b[i]*F[i] for i in range(s)])
        inc2 = dt*sum([rkm.bhat[i]*F[i] for i in range(s)])
        
        wr_unew = uu[-1] + inc1; E1_old = H_1(uu[-1]); E2_old = H_2(uu[-1])
        
        gammas, info, ier, mesg = fsolve(rgam,gammas0,args=(wr_unew,inc1,inc2,E1_old,E2_old),full_output=True)
        gamma1, gamma2 = gammas
        
        unew =  wr_unew + gamma1*inc1 + gamma2*inc2; t+=(1+gamma1+gamma2)*dt
        tt = np.append(tt, t)
        steps += 1
        uu = np.append(uu, np.reshape(unew.copy(), (1,len(unew))), axis=0) 
        G1 = np.append(G1, gamma1); G2 = np.append(G2, gamma2)
        print("Method = Relaxation %s: At step number = %d (time = %1.2f), integer flag = %d and γ1+γ2 = %f \n"%(Mthdname,steps,tt[-1],ier,gamma1+gamma2))

    return tt, uu, G1, G2

### Compute solutions by all the methods

In [ ]:
%time 
eqn = 'bi_H_LVS'
methods = [heun33,rk44,fehlberg45]
method_labels = ["Heun(3,3)", "RK(4,4)","Fehlberg(6,5)"]
method_names = ["Heuns3p3","RKs4p4","Fehlbergs6p5"]

# Inputs to solve the system of ODEs 
DT = [.04, .1, .1]; 
f = LVS3D_f; T = 400; 

# Initial condition
t0 = 0; u0 = np.array([1.0, 1.9, 0.5])
b_tt = []; b_uu = []; r_tt = []; r_uu = []  # empty list to store data by all the methods

for idx in range(len(methods)):
    print(idx)
    rkm = methods[idx]; dt = DT[idx]
    tt, uu, G1, G2 = compute_sol_multi_relaxation(method_labels[idx], rkm, dt, f, T, u0, t0)
    r_tt.append(tt); r_uu.append(uu)

for idx in range(len(methods)):
    rkm = methods[idx]; dt = DT[idx]
    tt, uu = compute_sol_without_relaxation(method_labels[idx], rkm, dt, f, T, u0, t0)
    b_tt.append(tt); b_uu.append(uu)

In [ ]:
import os
path = '%s'%('Figures')

import os
if not os.path.exists(path):
   os.makedirs(path)

### Compute and plot the changes in invariants by different methods

In [ ]:
b_H_1 = []; b_H_2 = []; r_H_1 = []; r_H_2 = []
for i in range(len(methods)):
    b_h_1 = [H_1(u) for u in b_uu[i]] - H_1(b_uu[i][0])
    b_h_2 = [H_2(u) for u in b_uu[i]] - H_2(b_uu[i][0])
    r_h_1 = [H_1(u) for u in r_uu[i]] - H_1(r_uu[i][0])
    r_h_2 = [H_2(u) for u in r_uu[i]] - H_2(r_uu[i][0])
    b_H_1.append(b_h_1); b_H_2.append(b_h_2)
    r_H_1.append(r_h_1); r_H_2.append(r_h_2)

In [ ]:
font = {#'family' : 'normal',
'weight' : 'normal',
'size'   : 14}
plt.rc('font', **font) 
plt.figure(figsize=(15, 4))
for i in range(len(methods)):
    plt.subplot(1,3,i+1)
    plt.plot(b_tt[i],b_H_1[i],':r',label="Baseline: $H_1(u_1(t),u_2(t),u_3(t))-H_1(u_1(0),u_2(0),u_3(0))$")
    plt.plot(r_tt[i],r_H_1[i],'-r',label="Relaxation: $H_1(u_1(t),u_2(t),u_3(t))-H_1(u_1(0),u_2(0),u_3(0))$")
    plt.plot(b_tt[i],b_H_2[i],':b',label="Baseline: $H_2(u_1(t),u_2(t),u_3(t))-H_2(u_1(0),u_2(0),u_3(0))$")
    plt.plot(r_tt[i],r_H_2[i],'-b',label="Relaxation: $H_2(u_1(t),u_2(t),u_3(t))-H_2(u_1(0),u_2(0),u_3(0))$")
    plt.title("%s with $\Delta t$ = %.2f"%(method_labels[i],DT[i]))
    plt.xlabel("$t$")
    plt.yscale("symlog", linthresh=1.e-14)
    plt.yticks([-1.e-6, -1.e-10, -1.e-14, 1.e-14, 1.e-10, 1.e-6])
    plt.tight_layout()
    
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper center', ncol=2, bbox_to_anchor=(0.5, 1.2))
#plt.show()
plt.savefig("./Figures/3DLVS_2_Inv_Error_Time.pdf", bbox_inches="tight")

### Proxy of exact solution by dp45 (Prince and Dormand) with absolute tolerence as 1e-16

In [ ]:
import numpy as np
from scipy import integrate 
from scipy.integrate import ode
# true solution by Dormand–Prince method
def LVS3D_f_tu(t,u):
    u1 = u[0]; u2 = u[1]; u3 = u[2];
    a = -1; b = -1; c = -1; la = 0; mu = 1; nu = -1;
    du1 = u1*(c*u2 + u3 + la)
    du2 = u2*(u1 + a*u3 + mu)
    du3 = u3*(b*u1 + u2 + nu)
    return np.array([du1, du2, du3])

def analytical_u_LVS(t,u0):
    true_u = np.zeros((len(t), len(u0)))  
    t0 = 0; true_u[0, :] = u0
    r = integrate.ode(LVS3D_f_tu).set_integrator("dopri5", rtol=1e-16, atol=1e-16)
    r.set_initial_value(u0, t0)
    for i in range(1, t.size):
        true_u[i, :] = r.integrate(t[i]) 
        if not r.successful():
            raise RuntimeError("Could not integrate")
    return true_u

### Compute and plot errors by different methods

In [ ]:
b_ERR = []; r_ERR = []; u0 = np.array([1.0, 1.9, 0.5])
for i in range(len(methods)):
    # maximum norm
    b_t = b_tt[i]; b_uexact = analytical_u_LVS(b_t,u0)
    b_err = np.max(np.abs(b_uu[i]-b_uexact),axis=1)
    r_t = r_tt[i]; r_uexact = analytical_u_LVS(r_t,u0)
    r_err = np.max(np.abs(r_uu[i]-r_uexact),axis=1)
    b_ERR.append(b_err); r_ERR.append(r_err)

In [ ]:
lgd_box_pos = [[0.4,0.2,0.5, 0.5],[0.4,0.1,0.5, 0.5],[0.4,0.5,0.5, 0.5]]
sl1_cons_mult = [9e-6,2e-5,8e-7]; sl1_p = [1,1,1]
sl2_cons_mult = [7*1e-6,6*1e-6,4e-7]; sl2_p = [2,2,2]
y_scale_line = [1e-10,1e-8,1e-7]

font = {#'family' : 'normal',
'weight' : 'normal',
'size'   : 14}
plt.rc('font', **font)
shift = 1
plt.figure(figsize=(15, 4))

for i in range(len(methods)):
    plt.subplot(1,3,i+1)
    plt.plot(b_tt[i]+shift,b_ERR[i],':',color='orangered',label="Baseline")
    plt.plot(r_tt[i]+shift,r_ERR[i],'-g',label="Relaxation")
    sl_t = np.linspace(10,T,1000)

    plt.plot(sl_t,sl1_cons_mult[i]*sl_t**sl1_p[i],'--',color='0.5',label="$\mathcal{O}(t^{%d})$"%(sl1_p[i]))
    plt.plot(sl_t,sl2_cons_mult[i]*sl_t**sl2_p[i],'-',color='0.5',label="$\mathcal{O}(t^{%d})$"%(sl2_p[i]))
     
    plt.xscale("log"); plt.yscale("log")
    plt.xlabel('t'); plt.ylabel('Error in q')
    plt.title("%s with $\Delta t$ = %.2f"%(method_labels[i],DT[i]))
    plt.tight_layout()
    
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper center', ncol=6, bbox_to_anchor=(0.5, 1.1))
plt.savefig("./Figures/3DLVS_Sol_Err_Time.pdf", bbox_inches="tight")